# 实现 Encoder Decoder问答系统
序列到序列的模型， 核心就是研究如何处理序列的问题。 

问题如下两个三位数的加法， 对答案进行预测

```
Q: 24+654
A: 678
```
  

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(1234)

print(tf.__version__)

1.13.1


### 定义网络

In [2]:


def inference(x, y, n_batch, is_training,
              input_digits=None,
              output_digits=None,
              n_hidden=None,
              n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)

    # Encode
    encoder = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    encoder = rnn.AttentionCellWrapper(encoder,
                                       input_digits,
                                       state_is_tuple=True)
    state = encoder.zero_state(n_batch, tf.float32)
    encoder_outputs = []
    encoder_states = []

    with tf.variable_scope('Encoder'):
        for t in range(input_digits):
            if t > 0:
                tf.get_variable_scope().reuse_variables()
            (output, state) = encoder(x[:, t, :], state)
            encoder_outputs.append(output)
            encoder_states.append(state)

    # Decode
    decoder = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    decoder = rnn.AttentionCellWrapper(decoder,
                                       input_digits,
                                       state_is_tuple=True)
    state = encoder_states[-1]
    decoder_outputs = [encoder_outputs[-1]]

    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    outputs = []

    with tf.variable_scope('Decoder'):
        for t in range(1, output_digits):
            if t > 1:
                tf.get_variable_scope().reuse_variables()

            if is_training is True:
                (output, state) = decoder(y[:, t-1, :], state)
            else:
                linear = tf.matmul(decoder_outputs[-1], V) + c
                out = tf.nn.softmax(linear)
                outputs.append(out)
                out = tf.one_hot(tf.argmax(out, -1), depth=output_digits)

                (output, state) = decoder(out, state)

            decoder_outputs.append(output)

    if is_training is True:
        output = tf.reshape(tf.concat(decoder_outputs, axis=1),
                            [-1, output_digits, n_hidden])

        linear = tf.einsum('ijk,kl->ijl', output, V) + c
        return tf.nn.softmax(linear)
    else:
        linear = tf.matmul(decoder_outputs[-1], V) + c
        out = tf.nn.softmax(linear)
        outputs.append(out)

        output = tf.reshape(tf.concat(outputs, axis=1),
                            [-1, output_digits, n_out])
        return output


def loss(y, t):
    cross_entropy = \
        tf.reduce_mean(-tf.reduce_sum(
                       t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)),
                       reduction_indices=[1]))
    return cross_entropy


def training(loss):
    optimizer = \
        tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)
    train_step = optimizer.minimize(loss)
    return train_step


def accuracy(y, t):
    correct_prediction = tf.equal(tf.argmax(y, -1), tf.argmax(t, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy




In [6]:

def n(digits=2):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)

# 序列补齐
def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))
print("n() = {}".format(n()))
chars = '897+2'
padding_chars = padding(chars, 10)
print("[{}] --> [{}]".format(chars, padding_chars))

n() = 37
[897+2] --> [897+2     ]


### 生成数据

In [4]:
N = 5000
N_train = int(N * 0.8)
N_validation = N - N_train

digits = 2  
input_digits = digits * 2 + 1  # 例： 1234+5678
output_digits = digits + 1  # 5000+5000 = 10000 

added = set()
questions = []
answers = []

chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

index = 0
while len(questions) < N:
    a, b = n(digits), n(digits)  
    index += 1
    pair = tuple(sorted((a, b)))
    if pair in added:
        continue


    if index > 1e+9:
        print("------- break")
        break
        
    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)  
    answer = str(a + b)
    answer = padding(answer, output_digits)  

    added.add(pair)
    questions.append(question)
    answers.append(answer)

In [5]:
print(questions[0:5])
print(answers[0:5])

['3+37 ', '35+4 ', '68+1 ', '7+81 ', '98+43']
['40 ', '39 ', '69 ', '88 ', '141']


### 划分训练测试数据集

In [6]:
X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), output_digits, len(chars)), dtype=np.integer)

print(X.shape)
for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X, Y, train_size=N_train)

(5000, 5, 12)


/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
print("X\n", X_train[0])
print("Y\n", Y_train[0])

X
 [[0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]]
Y
 [[0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]]


### 定义参数

In [8]:
tf.reset_default_graph()
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

x = tf.placeholder(tf.float32, shape=[None, input_digits, n_in])
t = tf.placeholder(tf.float32, shape=[None, output_digits, n_out])
n_batch = tf.placeholder(tf.int32, shape=[])
is_training = tf.placeholder(tf.bool)

y = inference(x, t, n_batch, is_training,
              input_digits=input_digits,
              output_digits=output_digits,
              n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

acc = accuracy(y, t)

history = {
    'val_loss': [],
    'val_acc': []
}


epochs = 200
batch_size = 256

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

n_batches = N_train // batch_size



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


### 进行训练

In [11]:
X_, Y_ = shuffle(X_train, Y_train)

for i in range(n_batches):
    start = i * batch_size
    end = start + batch_size

    sess.run(train_step, feed_dict={
        x: X_[start:end],
        t: Y_[start:end],
        n_batch: batch_size,
        is_training: True
    })

val_loss = loss.eval(session=sess, feed_dict={
    x: X_validation,
    t: Y_validation,
    n_batch: N_validation,
    is_training: False
})
val_acc = acc.eval(session=sess, feed_dict={
    x: X_validation,
    t: Y_validation,
    n_batch: N_validation,
    is_training: False
})

history['val_loss'].append(val_loss)
history['val_acc'].append(val_acc)
print('=' * 10, 'Epoch:', epoch, 'val_loss:', val_loss , 'val_acc: ', val_acc)

test_count = 10
correct_count =0 
for i in range(test_count):
    index = np.random.randint(0, N_validation)
    question = X_validation[np.array([index])]
    answer = Y_validation[np.array([index])]
    prediction = y.eval(session=sess, feed_dict={
        x: question,
        # t: answer,
        n_batch: 1,
        is_training: False
    })
    question = question.argmax(axis=-1)
    answer = answer.argmax(axis=-1)
    prediction = np.argmax(prediction, -1)

    q = ''.join(indices_char[i] for i in question[0])
    a = ''.join(indices_char[i] for i in answer[0])
    p = ''.join(indices_char[i] for i in prediction[0])


    result = False
    if a == p:
        result = True
        correct_count += 1

    print('{:5} -->  {:10s} = {}'.format(result, q, p))
print('  accuracy: {} '.format( float(correct_count/test_count)))



========== Epoch: 0 val_loss: 0.5088091 val_acc:  0.34066668
    0 -->  2+37       = 1  
    0 -->  43+55      = 1  
    0 -->  24+67      = 1  
    0 -->  13+97      = 1  
    0 -->  74+57      = 1  
    0 -->  47+50      = 1  
    0 -->  50+65      = 1  
    0 -->  30+61      = 1  
    0 -->  78+30      = 1  
    0 -->  35+48      = 1  
  accuracy: 0.0 
========== Epoch: 1 val_loss: 0.49130738 val_acc:  0.38266668
    0 -->  29+30      = 11 
    0 -->  9+83       = 11 
    0 -->  81+66      = 11 
    0 -->  71+49      = 11 
    0 -->  71+71      = 11 
    0 -->  60+24      = 11 
    0 -->  6+82       = 11 
    0 -->  80+10      = 11 
    0 -->  81+34      = 11 
    0 -->  47+70      = 11 
  accuracy: 0.0 
========== Epoch: 2 val_loss: 0.48247966 val_acc:  0.38633335
    0 -->  60+24      = 10 
    0 -->  79+58      = 11 
    0 -->  84+77      = 11 
    0 -->  67+84      = 11 
    0 -->  11+36      = 10 
    0 -->  30+19      = 11 
    0 -->  14+88      = 11 
    0 -->  9+54       = 1

In [12]:
val_acc = acc.eval(session=sess, feed_dict={
            x: X_validation,
            t: Y_validation,
            n_batch: N_validation,
            is_training: False
        })

In [13]:
val_acc

0.8146667